In [1]:
import re
#import required libraries
import math
import numpy as np
import pandas as pd
import re
import pyphonetics
import phonetics
import pydata_google_auth
import Monkey_Type_Detection as mtd
from arcgis.gis import GIS
from getpass import getpass
import skimpy
import wikipedia as wiki
import requests
from arcgis.geocoding import batch_geocode, Geocoder, get_geocoders, batch_geocode, geocode
from google_trans_new import google_translator

def get_file_ext(file_name):
    file_ext=file_name[len(file_name)-5:].rpartition('.')[1]
    return file_ext

def read_file(file_name,file_ext):
    #extract file name excluding the file extension -
    #add other possible file formats for data source here.
    file_name_without_ext=file_name.rpartition('.')[0]
    df=pd.DataFrame()
    if file_ext=='csv':
        df=pd.read_csv(file_name)
    elif file_ext=='pq':
        df=pd.read_parquet(file_name)
    elif file_ext=='pkl':
        df=pd.read_pickle(file_name)
    elif file_ext=='json':
        df=pd.read_json(file_name)

    else:
        raise Exception("Invalid file format...", file_ext)
    return df

def get_postal_pincode_data():
    
    from skimpy import clean_columns

    postal_pincode_data=pd.read_csv('postal_pincode_data.csv')
    messy_df = postal_pincode_data
    clean_df = clean_columns(messy_df)
    clean_df.columns.tolist()

def regex_match(pincode,pattern):
    if re.match(pincode, pattern):
        return True
    else:
        return False



def check_pincode_correct(pincode, address):
    pincode=str(pincode) #convert the input pincode - into string format, if not already - the first digit in pincode is zero - it will be removed, and pincode - will be read as 5 digits
    #check if it has 6 digits
    if len(pincode)<6:
        raise Exception("Invalid pincode - pincode contains less than 6 digits")
    elif len(pincode)>6:
        raise Exception("Pincode cannot have more than 6 digits...")
    elif pincode[0]=='0':
        raise Exception("First digit of pincode cannot be zero...")
    pincode_pattern=r"^[1-9]{1}[0-9]{2}\s{0,1}[0-9]{3}$"
    pincode_pattern_valid=regex_match(pincode_pattern,pincode)
    if (pincode_pattern_valid==False):
        raise Exception("Pincode Format not valid...")
    else: #entered pincode format is valid
        #now check if the entered pincode present in promise or postal database -
        postal_pincode_df=pd.read_csv('postal_pincode_data.csv')
        #convert both postal pincode and promise pincode list - to str types - to append both lists
        postal_pincode_df['pincode']=postal_pincode_df['pincode'].astype(str)
        postal_pincode_list=list(postal_pincode_df['pincode'].drop_duplicates().to_numpy())
        print("len of postal_pincode_list: ",len(postal_pincode_list))

        promise_pincode_df=pd.read_parquet('promise_state_name.pq')
        promise_pincode_df['pincode']=promise_pincode_df['pincode'].astype(str)
        promise_pincode_list=list(promise_pincode_df['pincode'].drop_duplicates().to_numpy())
        print("len of promise_pincode_list: ",len(promise_pincode_list))
        #get combined list of two pincodes -
        pincode_list=list(set(postal_pincode_list)|set(promise_pincode_list))
        print("len of pincode list: ",len(pincode_list))

        print("checking if pincode present in exhaustive list pincodes - db....")
        #check if pincode present in the overall pincode list -
        if pincode not in pincode_list:
            raise Exception("Pincode not found in database...")
        else:
            #now if the format of the entered pincode is valid and if it is also found in the overall pincode database...
            #Next step is to see if the pincode matches - the entered address
            #Two ways to check -
            # if the city matches pincode
            # if the locality or other smaller - division - mapped to a pincode - based on postal pincode data
            print("pincode entered is valid...")
            print("Pincode Address Match Check - yet to check if locality/address string matches with the entered pincode....")
            """
                        Postal data - attributes -
                        officename	pincode	officeType	Deliverystatus	divisionname regionname	circlename	Taluk	Districtname	statename	Telephone	Related Suboffice	Related Headoffice	longitude	latitude

            """
        pincode_expected_address_component_df=postal_pincode_df.loc[postal_pincode_df['pincode']==pincode,['officename','officeType','divisionname','regionname','circlename','Taluk','Districtname','statename','Related Suboffice','Related Headoffice','longitude','latitude']]
        postal_data_cols=['officename','officeType','divisionname','regionname','circlename','Taluk','Districtname','statename','Related Suboffice','Related Headoffice','longitude','latitude']
        for col in postal_data_cols:
            #convert dtype to str - for all the address_dataset part strings
            pincode_expected_address_component_df.loc[pincode_expected_address_component_df[col].isnull()==True,col]="None"
            pincode_expected_address_component_df[col]=pincode_expected_address_component_df[col].astype(str)
            #convert the address_dataset substr - to lower
            pincode_expected_address_component_df[col]=pincode_expected_address_component_df[col].str.lower()
            #get pincode's correspondong - address string - address
            pincode_address_mismatch=pincode_address_match_similarity(pincode,address,pincode_expected_address_component_df)

def pincode_address_match_similarity(pincode, address, expected_address):
    #check for expected address' similarity with actual address given for a pincode -

#to map address to the type - for eg. if an address corresponds to an apartment or individual building
        """
        1. solution 1 - use Google Maps API to see if the extracted door number of a specific street name corresponds to an apartment or a house - use Selenium - to extract the address details - corresponding to the search - if we need to avoid a geolocation API

        2. solution 2 - analyze the deliveries history for the same address (door number + street name) and if the total number of deliveries is significantly higher than the average, it should be an indicator that it refers to an apartment or office space, if it significantly lower it may point to it being a house
        """
def set_typology_feature_for_address(address):
    pass

#to classify an address - residential or a commercial/industrial area

"""
1. infer from address corresponding details - using geolocation API/extract addition location info - using selenium from maps.google.com - used for tagging area type -
2. analyze - the neigbouring addresses - if they are tagged - by area type and infer - area type - based on the nearest address info we have
"""
def set_area_feature_for_address(address):
    pass

def locality_set_for_given_pincode(pincode):
    """
    refer to ACT db - to get the range of localities a pincode is mapped to - along with
    only use delivered address examples without any address issue or pincode address mismatch issue identified - also
    use - courier partner's lat long and address lat long - from shopify or geolocation api mapping (used for training data)
    if the distance between the two lat long coordinates - less than a threshold -
    """
    pass


#clean data - remove missing, incorrect pincodes,duplicate pincodes
#get pincode, taluk mapping
def geolocation_mapping_fwd(x): #address to lat long mapping
    #using geopy, using arcgis, using position stack
    """ http://api.positionstack.com/v1/forward
        ? access_key ="""
    import requests
    #try getting location data from google maps
    response = requests.get('https://google.com/')
    print(response)
    """<Response [200]>
        & query = 1600 Pennsylvania Ave NW, Washington DC"""

def geolocation_mapping_reverse(x): #lat long to address -> address entity mappingdef composite_function(f, g):
        # return lambda x : f(g(x))
        pass

###
def pincode_validation(pincode,address):
    # pincode_format_valid=check_pincode_format(pincode)
    pincode_address_valid=check_pincode_correct(pincode,address)
    return pincode_address_valid

In [ ]:
pincode_expected_address_component_df=postal_pincode_df.loc[postal_pincode_df['pincode']==pincode,['officename','officeType','divisionname','regionname','circlename','Taluk','Districtname','statename','Related Suboffice','Related Headoffice','longitude','latitude']]
        postal_data_cols=['officename','officeType','divisionname','regionname','circlename','Taluk','Districtname','statename','Related Suboffice','Related Headoffice','longitude','latitude']
        for col in postal_data_cols:
            #convert dtype to str - for all the address_dataset part strings
            pincode_expected_address_component_df.loc[pincode_expected_address_component_df[col].isnull()==True,col]="None"
            pincode_expected_address_component_df[col]=pincode_expected_address_component_df[col].astype(str)
            #convert the address_dataset substr - to lower
            pincode_expected_address_component_df[col]=pincode_expected_address_component_df[col].str.lower()
            #get pincode's correspondong - address string - address
            pincode_address_mismatch=pincode_address_match_similarity(pincode,address,pincode_expected_address_component_df)


In [9]:
#checking for pyspark installation
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

#initiate spark context
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf=pyspark.SparkConf().setAppName('SparkApp').setMaster('local')
sc=pyspark.SparkContext(conf=conf)
spark=SparkSession(sc)  #??

#testing functionality

numeric_val=sc.parallelize([1,2,3])
numeric_val.map(lambda x: x**3).collect()

sc.stop() #to stop the current spark session


In [10]:
 osmDF = spark.read.format("osm.pbf").option("split", "false").load("india-latest.osm.pbf")

Py4JJavaError: An error occurred while calling o97.read.
: java.lang.IllegalStateException: LiveListenerBus is stopped.
	at org.apache.spark.scheduler.LiveListenerBus.addToQueue(LiveListenerBus.scala:97)
	at org.apache.spark.scheduler.LiveListenerBus.addToStatusQueue(LiveListenerBus.scala:80)
	at org.apache.spark.sql.internal.SharedState.<init>(SharedState.scala:115)
	at org.apache.spark.sql.SparkSession.$anonfun$sharedState$1(SparkSession.scala:139)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.SparkSession.sharedState$lzycompute(SparkSession.scala:139)
	at org.apache.spark.sql.SparkSession.sharedState(SparkSession.scala:138)
	at org.apache.spark.sql.SparkSession.$anonfun$sessionState$2(SparkSession.scala:158)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.SparkSession.sessionState$lzycompute(SparkSession.scala:156)
	at org.apache.spark.sql.SparkSession.sessionState(SparkSession.scala:153)
	at org.apache.spark.sql.DataFrameRea

In [2]:
#pseudo code for pincode validation using - Lat long - from delhivery, amazon

"""
preprocess - address strings
pick - locality names 

"""

'\npreprocess - address strings\npick - locality names \n\n'

In [3]:
pincode_validation("123456","HSR Layout")

len of postal_pincode_list:  19100
len of promise_pincode_list:  29129
len of pincode list:  29129
checking if pincode present in exhaustive list pincodes - db....


Exception: Pincode not found in database...

In [4]:
pincode_validation("600049","AnnaNagar")
#for eg. here pincode - entered - doesn't match the locality name

len of postal_pincode_list:  19100
len of promise_pincode_list:  29129
len of pincode list:  29129
checking if pincode present in exhaustive list pincodes - db....
pincode entered is valid...
Pincode Address Match Check - yet to check if locality/address string matches with the entered pincode....


In [6]:
pincode_pattern=r"^[1-9]{1}[0-9]{2}\s{0,1}[0-9]{3}$"
regex_match('600 049',pincode_pattern)

False

In [ ]:
=from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="<<some_app_name>>")
location = geolocator.geocode("")

In [1]:
import pandas as pd
import geopy

geopy.geocoders.options.default_user_agent = "myMAil"

def get_zipcode(subsmi, geolocator, lat_field, lon_field):
    print(subsmi)
    print(type(subsmi))
    location = geolocator.reverse((subsmi[lat_field], subsmi[lon_field]))
    return location.raw['address']['postcode']


geolocator = geopy.Nominatim()
df = pd.DataFrame({
    'lat': [29.39291, 29.39923, 29.40147, 29.38752, 29.39291, 29.39537, 29.39343, 29.39291, 29.39556],
    'lon': [-98.50925, -98.51256, -98.51123, -98.52372, -98.50925, -98.50402, -98.49707, -98.50925, -98.53148]
})

zipcodes = df.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='lat', lon_field='lon')

lat    29.39291
lon   -98.50925
Name: 0, dtype: float64
<class 'pandas.core.series.Series'>
lat    29.39923
lon   -98.51256
Name: 1, dtype: float64
<class 'pandas.core.series.Series'>
lat    29.40147
lon   -98.51123
Name: 2, dtype: float64
<class 'pandas.core.series.Series'>
lat    29.38752
lon   -98.52372
Name: 3, dtype: float64
<class 'pandas.core.series.Series'>
lat    29.39291
lon   -98.50925
Name: 4, dtype: float64
<class 'pandas.core.series.Series'>
lat    29.39537
lon   -98.50402
Name: 5, dtype: float64
<class 'pandas.core.series.Series'>
lat    29.39343
lon   -98.49707
Name: 6, dtype: float64
<class 'pandas.core.series.Series'>
lat    29.39291
lon   -98.50925
Name: 7, dtype: float64
<class 'pandas.core.series.Series'>
lat    29.39556
lon   -98.53148
Name: 8, dtype: float64
<class 'pandas.core.series.Series'>


In [9]:
from arcgis.geocoding import reverse_geocode
from arcgis.geometry import Geometry
from arcgis.gis import GIS
from getpass import getpass
import pandas as pd
password=getpass()
gis = GIS("http://www.arcgis.com", "jarvisvb", password)

def get_zip(df, lon_field, lat_field):
    location = reverse_geocode((Geometry({"x":float(df[lon_field]), "y":float(df[lat_field]), "spatialReference":{"wkid": 4326}})))
    print("location: ",location)
    return location['address']['Postal']

df = pd.DataFrame({
    'Lat': [29.39291, 29.39923, 29.40147, 29.38752, 29.39291, 29.39537, 29.39343, 29.39291, 29.39556],
    'Lon': [-98.50925, -98.51256, -98.51123, -98.52372, -98.50925, -98.50402, -98.49707, -98.50925, -98.53148]
})

zipcodes_argis = df.apply(get_zip, axis=1, lat_field='Lat', lon_field='Lon')

········
location:  {'address': {'Match_addr': 'Burbank High School', 'LongLabel': 'Burbank High School, 1002 Edwards St, San Antonio, TX, 78204, USA', 'ShortLabel': 'Burbank High School', 'Addr_type': 'POI', 'Type': 'School', 'PlaceName': 'Burbank High School', 'AddNum': '1002', 'Address': '1002 Edwards St', 'Block': '', 'Sector': '', 'Neighborhood': 'Englewood', 'District': '', 'City': 'San Antonio', 'MetroArea': 'San Antonio Metro Area', 'Subregion': 'Bexar County', 'Region': 'Texas', 'RegionAbbr': 'TX', 'Territory': '', 'Postal': '78204', 'PostalExt': '', 'CntryName': 'United States', 'CountryCode': 'USA'}, 'location': {'x': -98.50925, 'y': 29.39291, 'spatialReference': {'wkid': 4326, 'latestWkid': 4326}}}
location:  {'address': {'Match_addr': 'Viva Vegeria', 'LongLabel': 'Viva Vegeria, 1422 Nogalitos St, San Antonio, TX, 78204, USA', 'ShortLabel': 'Viva Vegeria', 'Addr_type': 'POI', 'Type': 'Fast Food', 'PlaceName': 'Viva Vegeria', 'AddNum': '1422', 'Address': '1422 Nogalitos St',

In [4]:
zipcodes

0    78204
1    78204
2    78204
3    78225
4    78204
5    78204
6    78204
7    78204
8    78225
dtype: object

In [12]:
import geopy
import pandas as pd


def get_zipcode(df, geolocator, lat_field, lon_field):
    location = geolocator.reverse((df[lat_field], df[lon_field]))
    print("location: ",location)
    print("location.raw: ",location.raw['address'])
    return location.raw['address']['postcode']

def reverse_geocode_raw_address(df, geolocator, lat_field, lon_field):
    location = geolocator.reverse((df[lat_field], df[lon_field]))
    print("location: ",location)
    print("location.raw: ",location.raw['address'])
    return location.raw['address']

def reverse_geocode_location(df, geolocator, lat_field, lon_field):
    location = geolocator.reverse((df[lat_field], df[lon_field]))
    print("location: ",location)
    print("location.raw: ",location.raw['address'])
    return location
    


geolocator = geopy.Nominatim(user_agent='http')
geopy.geocoders.options.default_user_agent = "my-application"
# nom=Nominatim(user_agent="http")
"""
df = pd.DataFrame({
    'Lat': [29.39291, 29.39923, 29.40147, 29.38752, 29.39291, 29.39537, 29.39343, 29.39291, 29.39556],
    'Lon': [-98.50925, -98.51256, -98.51123, -98.52372, -98.50925, -98.50402, -98.49707, -98.50925, -98.53148]
})"""
df=pd.read_parquet('open_street_map_data/overall_osm_df.pq')
df['reverse_geocode_zip']=np.nan
df['reverse_geocode_location']=np.nan
df['reverse_geocode_raw_address']=np.nan
df=df.reset_index()
df['latitude']=df['latitude'].astype(float)
df['longitude']=df['longitude'].astype(float)

df['reverse_geocode_zip'] = df.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='latitude', lon_field='latitude')
df['reverse_geocode_raw_address']=df.apply(reverse_geocode_raw_address,geolocator=geolocator,lat_field='latitude',lon_field='longitude')
df['reverse_geocode_location']=df.apply(reverse_geocode_location,geolocator=geolocator,lat_field='latitude',lon_field='longitude')

#reverse geocoded address 
df.head()

location:  None


AttributeError: 'NoneType' object has no attribute 'raw'

In [35]:

df_new = pd.DataFrame({
    'latitude': [76.333187],
    'longitude': [10.306511]
})
location=reverse_geocode_location(df_new,geolocator=geolocator,lat_field='latitude',lon_field='longitude')
print(location)

ValueError: Must be a coordinate pair or Point

In [37]:

df = pd.DataFrame({
    'Lat': [29.39291, 29.39923, 29.40147, 29.38752, 29.39291, 29.39537, 29.39343, 29.39291, 29.39556],
    'Lon': [-98.50925, -98.51256, -98.51123, -98.52372, -98.50925, -98.50402, -98.49707, -98.50925, -98.53148]
})
df

,Lat,Lon
0,29.39291,-98.50925
1,29.39923,-98.51256
2,29.40147,-98.51123
3,29.38752,-98.52372
4,29.39291,-98.50925
5,29.39537,-98.50402
6,29.39343,-98.49707
7,29.39291,-98.50925
8,29.39556,-98.53148


In [38]:
df['Lat'].iloc[0]

29.39291

In [36]:
df_new

,latitude,longitude
0,76.333187,10.306511


In [29]:
df.loc[df['reverse_geocode_zip'].isnull()==False]

,latitude,longitude,name,tags,place,amenity,postcode,source,state,full,...,animal,ford,gate,contact,check_date:internet_access,n_dt2011_0,source:church,reverse_geocode_zip,reverse_geocode_location,reverse_geocode_raw_address


In [33]:
df[['latitude','longitude']].iloc[0]

latitude     76.333187
longitude    10.306511
Name: 0, dtype: float64

In [32]:
df

,index,latitude,longitude,name,tags,place,amenity,postcode,source,state,...,animal,ford,gate,contact,check_date:internet_access,n_dt2011_0,source:church,reverse_geocode_zip,reverse_geocode_location,reverse_geocode_raw_address
0,0,76.333187,10.306511,Muncipal Hq,"{'highway': 'bus_stop', 'name': 'Muncipal Hq'}",None,None,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
1,0,76.338491,10.307961,Chalakudy North Junction,"{'junction': 'yes', 'name': 'Chalakudy North Junction'}",None,None,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
2,0,76.331453,10.305761,ITI Junction,"{'highway': 'bus_stop', 'name': 'ITI Junction'}",None,None,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
3,0,76.337068,10.310996,Anamala Junction,"{'junction': 'yes', 'name': 'Anamala Junction'}",None,None,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
4,0,76.318922,10.301338,Ambala nada,"{'highway': 'bus_stop', 'name': 'Ambala nada'}",None,None,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170995,0,88.380188,26.723450,Dolphin Hotel,"{'addr:street': 'Matighadha Mor', 'name': 'Dolphin Hotel', 'name:en': 'Dolphin Hotel', 'tourism': 'hotel'}",None,None,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
170996,0,79.053974,21.106030,Jetwan,"{'name': 'Jetwan', 'place': 'neighbourhood'}",neighbourhood,None,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
170997,0,72.875659,18.639850,Koliwada,"{'name': 'Koliwada', 'place': 'neighbourhood'}",neighbourhood,None,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
170998,0,72.880915,18.647413,MHADA Colony,"{'name': 'MHADA Colony', 'place': 'neighbourhood'}",neighbourhood,None,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN


In [18]:
import numpy as np


In [ ]:
def fwd_geocoding(df)

In [8]:
zipcodes_geopy

0    78204
1    78204
2    78204
3    78225
4    78204
5    78204
6    78204
7    78204
8    78225
dtype: object

In [ ]:
osm